## Prepare Data

In [1]:
import re
import os
import pandas as pd

regex = re.compile(r'\[.+\]\n', re.IGNORECASE)
file_paths, file_names, sessions, emotions = [], [], [], []
emotion_map = {'neu': 'neutral', 'ang': 'angry', 'hap': 'happy', 'exc': 'happy', 'sad': 'sad'}

for session in range(1, 6):
    emo_evaluation_dir = f'/home/jz3313/IEMOCAP_full_release/Session{session}/dialog/EmoEvaluation/'
    file_dir = f'/home/jz3313/IEMOCAP_full_release/Session{session}/sentences/wav/'
    evaluation_files = [l for l in os.listdir(emo_evaluation_dir) if 'Ses' in l]
    for file in evaluation_files:
        with open(emo_evaluation_dir + file) as f:
            content = f.read()
        lines = re.findall(regex, content)
        for line in lines[1:]:  # the first line is a header
            start_end_time, wav_file_name, emotion, val_act_dom = line.strip().split('\t')
            dir_name = '_'.join(wav_file_name.split('_')[:-1])
            if emotion in emotion_map:
                file_paths.append(file_dir+dir_name+'/'+wav_file_name+'.wav')
                file_names.append(wav_file_name)
                sessions.append(session)
                emotions.append(emotion_map[emotion])

In [2]:
file = pd.DataFrame({'path':file_paths, 'name': file_names, 'session': sessions, 'emotion': emotions})

In [3]:
file.head()

,path,name,session,emotion
0,/home/jz3313/IEMOCAP_full_release/Session1/sen...,Ses01F_impro02_F000,1,sad
1,/home/jz3313/IEMOCAP_full_release/Session1/sen...,Ses01F_impro02_F001,1,sad
2,/home/jz3313/IEMOCAP_full_release/Session1/sen...,Ses01F_impro02_F002,1,sad
3,/home/jz3313/IEMOCAP_full_release/Session1/sen...,Ses01F_impro02_F003,1,neutral
4,/home/jz3313/IEMOCAP_full_release/Session1/sen...,Ses01F_impro02_F004,1,sad


In [4]:
import torch
import torchaudio

print(torch.__version__)
print(torchaudio.__version__)

torch.random.manual_seed(0)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(device)

1.12.1+cu113
0.12.1+cu113
cuda


In [5]:
bundle = torchaudio.pipelines.WAV2VEC2_BASE
extractor = bundle.get_model()
print(extractor.__class__)

<class 'torchaudio.models.wav2vec2.model.Wav2Vec2Model'>


In [ ]:
from tqdm import tqdm

for _, row in tqdm(file.iterrows()):
    path, name, session, emotion = row[0], row[1], row[2], row[3]
    wave, sr = torchaudio.load(path)
    if sr != bundle.sample_rate:
        wave = torchaudio.functional.resample(wave, sr, bundle.sample_rate)
    with torch.inference_mode():
        feature, _ = extractor.extract_features(wave)
    feature = [f[0] for f in feature]
    feature = torch.stack(feature)
    save_path = f'../data/wav2vecbase/Session{session}/{name}.pt'
    torch.save(feature, save_path)

## Load Data

In [6]:
file['newpath'] = file.apply(lambda x: f'../data/wav2vecbase/Session{x[2]}/{x[1]}.pt', axis=1)

In [7]:
file.head()

,path,name,session,emotion,newpath
0,/home/jz3313/IEMOCAP_full_release/Session1/sen...,Ses01F_impro02_F000,1,sad,../data/wav2vecbase/Session1/Ses01F_impro02_F0...
1,/home/jz3313/IEMOCAP_full_release/Session1/sen...,Ses01F_impro02_F001,1,sad,../data/wav2vecbase/Session1/Ses01F_impro02_F0...
2,/home/jz3313/IEMOCAP_full_release/Session1/sen...,Ses01F_impro02_F002,1,sad,../data/wav2vecbase/Session1/Ses01F_impro02_F0...
3,/home/jz3313/IEMOCAP_full_release/Session1/sen...,Ses01F_impro02_F003,1,neutral,../data/wav2vecbase/Session1/Ses01F_impro02_F0...
4,/home/jz3313/IEMOCAP_full_release/Session1/sen...,Ses01F_impro02_F004,1,sad,../data/wav2vecbase/Session1/Ses01F_impro02_F0...


In [8]:
holdout = 1
train = file[file['session'] != holdout]
test = file[file['session'] == holdout]

In [9]:
class MyDataSet(torch.utils.data.Dataset):
    def __init__(self, datas, labels, label_transform):
        super(MyDataSet).__init__()
        self.datas = datas
        self.labels = labels
        self.label_transform = label_transform
        
    def __getitem__(self, idx):
        audio = self.datas[idx]
        label = self.label_transform[self.labels[idx]]
        length = audio.size(1)
        return audio, length, label
    
    def __len__(self):
        return len(self.labels)

In [10]:
def collate_indic(data):
    audios, lengths, labels = zip(*data)
    max_len = max(lengths)
    n_ftrs = audios[0].size(2)
    n_dims = audios[0].size(0)
    features = torch.zeros((len(audios), n_dims, max_len, n_ftrs))
    labels = torch.tensor(labels)
    lengths = torch.tensor(lengths)

    for i in range(len(data)):
        j, k = audios[i].size(1), audios[i].size(2)
        features[i] = torch.cat([audios[i], torch.zeros((n_dims, max_len - j, k))], dim=1)

    return features, lengths, labels

In [11]:
categories = ['angry', 'happy', 'neutral', 'sad']
cate_dic = {}
for i, cate in enumerate(categories):
    cate_dic[cate] = i
cate_dic

{'angry': 0, 'happy': 1, 'neutral': 2, 'sad': 3}

## Train with 3CNN+LSTM

In [12]:
import torch.nn as nn
import torch.nn.functional as F

class ICASSP3CNN(nn.Module):
    def __init__(self, vocab_size, dims = 12, embed_size=128, hidden_size=512, num_lstm_layers = 2, bidirectional = False, label_size=7):
        super().__init__()
        self.n_layers = num_lstm_layers 
        self.hidden = hidden_size
        self.bidirectional = bidirectional
        
        self.aggr = nn.Conv1d(in_channels=dims, out_channels=1, kernel_size=1)
        
        self.embed = nn.Linear(in_features = vocab_size, out_features = embed_size)

        self.cnn  = nn.Conv1d(embed_size, embed_size, kernel_size=3, padding=1)
        self.cnn2 = nn.Conv1d(embed_size, embed_size, kernel_size=5, padding=2)
        self.cnn3 = nn.Conv1d(embed_size, embed_size, kernel_size=7, padding=3)

        self.batchnorm = nn.BatchNorm1d(3 * embed_size)

        self.lstm = nn.LSTM(input_size = 3 * embed_size, 
                            hidden_size = hidden_size, 
                            num_layers = num_lstm_layers, 
                            bidirectional = bidirectional)

        self.linear = nn.Linear(in_features = 2 * hidden_size if bidirectional else hidden_size, 
                                out_features = label_size)


    def forward(self, x, lengths):
        """
        padded_x: (B,T) padded LongTensor
        """
        n, d, b, t = x.size(0), x.size(1), x.size(2), x.size(3)
        x = torch.flatten(x, start_dim=2)
        input = self.aggr(x)
        input = torch.reshape(input, (n, b, t))
        input = self.embed(input)

        batch_size = input.size(0)
        input = input.transpose(1,2)    # (B,T,H) -> (B,H,T)

        cnn_output = torch.cat([self.cnn(input), self.cnn2(input), self.cnn3(input)], dim=1)

        input = F.relu(self.batchnorm(cnn_output))

        input = input.transpose(1,2)

        pack_tensor = nn.utils.rnn.pack_padded_sequence(input, lengths, batch_first=True, enforce_sorted=False)
        _, (hn, cn) = self.lstm(pack_tensor)

        if self.bidirectional:
            h_n = hn.view(self.n_layers, 2, batch_size, self.hidden)
            h_n = torch.cat([ h_n[-1, 0,:], h_n[-1,1,:] ], dim = 1)
        else:
            h_n = hn[-1]

        logits = self.linear(h_n)

        return logits

### Model Traning on each layer 

In [13]:
from tqdm import tqdm
from torch.utils.data import DataLoader

traindata = []
for _, row in tqdm(train.iterrows()):
    traindata.append(torch.load(row[4]))

train_dataset = MyDataSet(traindata, train['emotion'].tolist(), cate_dic)
trainloader_args = dict(batch_size=64, shuffle=True)
train_dataloader = DataLoader(train_dataset, **trainloader_args, 
                              collate_fn=collate_indic)

4446it [05:42, 12.99it/s]


In [15]:
train_dataset = MyDataSet(traindata, train['emotion'].tolist(), cate_dic)
trainloader_args = dict(batch_size=64, shuffle=True)
train_dataloader = DataLoader(train_dataset, **trainloader_args, 
                              collate_fn=collate_indic)

In [16]:
from tqdm import tqdm
from torchsummary import summary
import torch.optim as optim

model = ICASSP3CNN(768)
model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [17]:
epochs = 50
train_losses = []
train_accuracies = []
valid_losses = []
valid_accuracies = []

for epoch in tqdm(range(epochs)):
    train_loss = 0
    acc_cnt = 0
    err_cnt = 0
    batch_cnt = 0
    model.train()
    for batch, (x, length, y) in enumerate(train_dataloader):
        x = x.to(device)
        y = y.to(device)
        optimizer.zero_grad()
        logits = model(x, length)
        loss = criterion(logits, y)
        loss.backward()
        optimizer.step()
        train_loss += loss.cpu().item()

        #model outputs
        out_val, out_indices = torch.max(logits, dim=1)
        tar_indices = y

        for i in range(len(out_indices)):
            if out_indices[i] == tar_indices[i]:
                acc_cnt += 1
            else:
                err_cnt += 1
        batch_cnt += 1
    
    train_loss = train_loss/batch_cnt
    train_accuracy = acc_cnt/(acc_cnt+err_cnt)
    train_accuracies.append(train_accuracy)
    train_losses.append(train_loss)
    
    print(f"epoch:{epoch+1}, train accu:{train_accuracy:.4f},", f"train loss:{train_loss:.2f}")

  2%|███▎                                                                                                                                                                  | 1/50 [01:35<1:17:35, 95.01s/it]

epoch:1, train accu:0.4593, train loss:1.15


  4%|██████▋                                                                                                                                                               | 2/50 [03:05<1:13:58, 92.47s/it]

epoch:2, train accu:0.5493, train loss:1.01


  6%|█████████▉                                                                                                                                                            | 3/50 [04:36<1:11:58, 91.89s/it]

...

epoch:48, train accu:0.9996, train loss:0.00


 98%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▋   | 49/50 [1:13:23<01:29, 89.41s/it]

epoch:49, train accu:1.0000, train loss:0.00


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [1:14:51<00:00, 89.83s/it]

epoch:50, train accu:1.0000, train loss:0.00


## Model Test

In [18]:
from tqdm import tqdm

testdata = []
for _, row in tqdm(test.iterrows()):
    testdata.append(torch.load(row[4]))
    
test_dataset = MyDataSet(testdata, test['emotion'].tolist(), cate_dic)
testloader_args = dict(batch_size=1, shuffle=True)
test_dataloader = DataLoader(test_dataset, **testloader_args, 
                             collate_fn=collate_indic)

1085it [01:31, 11.84it/s]


In [19]:
test_loss = 0
acc_cnt = 0
err_cnt = 0
batch_cnt = 0
model.eval()

for x, lengths, y in test_dataloader:

    x = x.to(device)
    y = y.to(device)

    logits = model(x, lengths)
    loss = criterion(logits, y)
    test_loss += loss.cpu().item()

    out_val, out_indices = torch.max(logits, dim=1)
    tar_indices = y

    for i in range(len(out_indices)):
        if out_indices[i] == tar_indices[i]:
            acc_cnt += 1
        else:
            err_cnt += 1
    batch_cnt += 1

test_loss = test_loss/batch_cnt
test_accuracy = acc_cnt/(acc_cnt+err_cnt)
print(f'test accuracy: {test_accuracy}')

test accuracy: 0.6092165898617512


In [23]:
for name, param in model.named_parameters():
    if param.requires_grad:
        print(name, param.data)

aggr.weight tensor([[[-0.1310],
         [ 0.1772],
         [ 0.2437],
         [ 0.0489],
         [ 0.1233],
         [-0.2575],
         [-0.3244],
         [ 0.0229],
         [ 0.0835],
         [-0.0186],
         [-0.0222],
         [ 0.0371]]], device='cuda:0')
aggr.bias tensor([0.0086], device='cuda:0')
embed.weight tensor([[ 0.0075, -0.0759, -0.0275,  ..., -0.0115,  0.1392,  0.1030],
        [ 0.0774,  0.0207,  0.0966,  ...,  0.0189,  0.0675,  0.0912],
        [ 0.0188,  0.0476, -0.0616,  ...,  0.0615,  0.0772,  0.0852],
        ...,
        [ 0.0554, -0.1219,  0.0362,  ..., -0.0379,  0.0833,  0.0365],
        [ 0.0158,  0.0221, -0.0307,  ..., -0.0336,  0.0295, -0.1591],
        [ 0.0599, -0.0022, -0.0519,  ...,  0.0911, -0.0293,  0.0891]],
       device='cuda:0')
embed.bias tensor([-0.0146,  0.0433, -0.0223, -0.0170, -0.0170,  0.0077, -0.0685,  0.0018,
         0.0197, -0.0623, -0.0254, -0.0841,  0.0169,  0.0671,  0.0249, -0.0353,
         0.0423,  0.0065, -0.0815,  0.0497,

In [20]:
model_path = f'../models/wav2vecbase/holdout{holdout}.pth'

torch.save({'epoch':epochs,
            'model_state_dict':model.state_dict(),
            'optimizer_state_dict':optimizer.state_dict()},
            model_path)